## 1 Carga de Librería, importación y limpieza de datos

In [1]:
# Comenzamos importando librerías para todos los casos de Uso
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
# Cargamos de Datos procedentes de dataset escrapeado y limpio (Dataset=Competitions_limpio)
competiciones=pd.read_csv('Competitions_limpio.csv')

In [3]:
# Ver estructura de tados de fichero competiciones
competiciones.head(1)

,Unnamed: 0,date,description,prize,tags,teams,title,topic
0,0,2014-09-16,Use the ATLAS experiment to identify the Higgs...,13000,NaN,1785,Higgs Boson Machine Learning Challenge,Featured


In [4]:
# Tamaño de nuestro dataset Competiciones
competiciones.shape

(329, 8)

In [5]:
# Elimnamos columna poco interesantes para serie temporal
competiciones=competiciones.drop(['description', 'prize', 'tags','topic'], axis=1)
competiciones.head(2)

,Unnamed: 0,date,teams,title
0,0,2014-09-16,1785,Higgs Boson Machine Learning Challenge
1,1,2013-09-29,80,See Click Predict Fix - Hackathon


In [6]:
# Revisamos los datos nulos para verificar si hay que realizar limpieza de datos
print("Contandor de datos nulos por columnas :",competiciones.isnull().any().any())     
print(competiciones.isnull().sum())

Contandor de datos nulos por columnas : False
Unnamed: 0    0
date          0
teams         0
title         0
dtype: int64


In [7]:
# Revisión Formato de cada uno de los campos
competiciones.dtypes

Unnamed: 0     int64
date          object
teams          int64
title         object
dtype: object

In [8]:
# Cambiamos la variable date a tipo datetime64 
competiciones['date'] = competiciones['date'].astype(datetime)

In [9]:
# Extraemos , del campo date, los datos de año y mes en formato YYYY-MM.
competiciones['date'] = competiciones['date'].str[:7]
competiciones.head(3)

,Unnamed: 0,date,teams,title
0,0,2014-09,1785,Higgs Boson Machine Learning Challenge
1,1,2013-09,80,See Click Predict Fix - Hackathon
2,2,2018-08,653,TrackML Particle Tracking Challenge


## 2 Creacion de Series Temporales
Hemos decidido, para su posible posterior uso en otros analisis, crear tres dataframe con tres series de tiempo
- Serie mensual de equipos participantes en competiciones
- Serie mensual de competiciones 
- Serie bivariante conteniendo los datos de equipos y competiciones

In [14]:
# Arreglo para generar periodo de ambito de la serie, desde 2010-05 a 2019-08
rango1 = pd.date_range(start='5/2010', end='09/2019', freq='M')
df = pd.DataFrame(rango1, columns=['date'])
df.head(2)

,date
0,2010-05-31
1,2010-06-30


In [15]:
df.shape

(112, 1)

In [16]:
# Realizamos adaptación a fecha corta del dataframe df 
df['date'] = df['date'].astype(str)
df['date'] = df['date'].str[:7]
df.head(2)

,date
0,2010-05
1,2010-06


In [17]:
# Primera Serie : Realizamos un agrupamiento por Date para calcular el numero de competiciones por mes y cambiamos nombres columnas
seriecompeticiones= competiciones[['title','date']].groupby('date').count()
serieteams= competiciones[['date','teams']].groupby('date').sum()
serieteams.reset_index(level=0, inplace=True)
serieteams = serieteams.rename(columns={'date':'date'})
serieteams.head(2)


,date,teams
0,2010-05,22
1,2010-06,63


- Hemos obtenido el numero de equipos que han participado, mes a mes, en las competiciones de Kaggle


In [18]:
# SegundaSerie : Realizamos un agrupamiento por Date  para calcular el numero de equipos que compiten por mes
seriecompeticiones= competiciones[['title','date']].groupby('date').count()
seriecompeticiones.reset_index(level=0, inplace=True)
seriecompeticiones = seriecompeticiones.rename(columns={'title':'Competiciones'})
seriecompeticiones.head(2)


,date,Competiciones
0,2010-05,1
1,2010-06,1


Hemos obtenido con el numero de competiciones mes a mes



In [19]:
# Tercera Serie : Realizamos union de seriecompeticiones y serie teams, para obetener una serie bivariable
seriebivariable=pd.merge(serieteams, seriecompeticiones, on='date', how='inner')


In [22]:
seriebivariable.tail(8)


,date,teams,Competiciones
98,2019-05,3730,2
99,2019-06,11054,10
100,2019-07,3715,2
101,2019-08,879,2
102,2019-10,57,2
103,2020-01,3314,1
104,2029-12,38,1
105,2100-12,18876,3


- Hemos obtenido la serie bivariante con el numero de competiciones y equipos mes a mes

## Competado de series temporales con toda la escala de tiempo e interpolación lineal de valores faltantes

In [23]:
# Serie Competiciones:Una vez estructurado las tres series, debemos completar los periodos haciendo join con dataframe
seriecompeticiones=pd.merge(df, seriecompeticiones,left_on='date',right_on='date', how='left')
seriecompeticiones.sort_values(by='date', ascending=True)
seriecompeticiones.head(2)

# Realizaremos, para los meses en los que no tenemos datos una interpolación lineal.

seriecompeticiones['Competiciones']=seriecompeticiones['Competiciones'].interpolate(method='linear',limit_direction ='backward', limit=1)
seriecompeticiones.head(20)


,date,Competiciones
0,2010-05,1.000000
1,2010-06,1.000000
2,2010-07,1.000000
3,2010-08,1.000000
4,2010-09,1.000000
5,2010-10,1.000000
6,2010-11,2.000000
7,2010-12,1.500000
8,2011-01,1.000000
9,2011-02,3.000000


In [24]:
# Serie Teams:Una vez estructurado las tres series, debemos completar los periodos haciendo join con dataframe
serieteams=pd.merge(df, serieteams,left_on='date',right_on='date', how='left')
serieteams.sort_values(by='date', ascending=True)
# Realizaremos, para los meses en los que no tenemos datos una interpolación lineal.
serieteams['teams']=serieteams['teams'].interpolate(method='linear',limit_direction ='backward', limit=1)
serieteams.head(5)


,date,teams
0,2010-05,22.0
1,2010-06,63.0
2,2010-07,85.0
3,2010-08,107.0
4,2010-09,55.0


In [768]:
# Serie bivariable: Para las series temporales hay que definir la fecha como indice... así es como lo haríamos para las tres series

seriebivariable=pd.merge(df, seriebivariable,left_on='date',right_on='date', how='left')
seriebivariable.head()
# Realizaremos, para los meses en los que no tenemos datos una interpolación lineal.
seriebivariable['teams']=seriebivariable['teams'].interpolate(method='linear',limit_direction ='backward', limit=1)
seriebivariable['Competiciones']=seriebivariable['Competiciones'].interpolate(method='linear',limit_direction ='backward', limit=1)
seriebivariable.head()

,date,teams,Competiciones
0,2010-05,22.0,1.0
1,2010-06,63.0,1.0
2,2010-07,85.0,1.0
3,2010-08,107.0,1.0
4,2010-09,55.0,1.0


# Limpieza final de columnas para exportación de series

In [773]:
seriecompeticiones = seriecompeticiones.rename(columns={'date':'Mes'})
serieteams = serieteams.rename(columns={'date':'Mes'})
seriebivariable = seriebivariable.rename(columns={'date':'Mes'})
seriecompeticiones.head()
seriecompeticiones.set_index('Mes',inplace=True)
serieteams.set_index('Mes',inplace=True)
seriebivariable.set_index('Mes',inplace=True)


In [775]:

seriecompeticiones.head()
serieteams.head()

,teams
Mes,
2010-05,22.0
2010-06,63.0
2010-07,85.0
2010-08,107.0
2010-09,55.0


In [776]:
# Convertimos las columnas teams y competiciones de todas las series a enteros
#seriecompeticiones['Competiciones'] = seriecompeticiones['Competiciones'].astype(int)
#serieteams['teams'] = serieteams['teams'].astype(int)
#seriebivariable['teams'] = seriebivariable['teams'].astype(int)
#seriebivariable['Competeciones'] = seriebivariable['Competiciones'].astype(int)

In [777]:
# Exportamos las tres series a sus correspondientes CSV para su posterior tratamiento
seriecompeticiones.to_csv('seriecompeticiones.csv')
serieteams.to_csv('serieteams.csv')
seriebivariable.to_csv('seriebivariable.csv')
df_completo.to_csv('seriecompleta.csv')

## FIN PREPARACION DATASETS